In [ ]:
# Mount the google drive with your google collab file.
from google.colab import drive
drive.mount('/content/drive') 

In [ ]:
# Import necessary packages
import tensorflow as tf
from tensorflow.python.keras import backend as K 
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.models import Model 
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D 
from tensorflow.python.keras.layers import Activation, Dropout, Flatten, Dense
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import tensorflow as tf
from tensorflow.keras import optimizers

In [ ]:
"""
Copy the training and validation data to the runtime memory and extract those files for
further processing.
"""
#!mkdir FinalData/
#!cp /content/drive/My\ Drive/Competitive/Dataset/augmented_2_3.zip FinalData/
#!unzip FinalData/augmented_2_3.zip -d FinalData/

#!mkdir ValData/
#!cp  /content/drive/My\ Drive/Competitive/Dataset/TestData.zip ValData/
#!unzip ValData/TestData.zip -d ValData/


train_data_dir = "/content/FinalData/augmented_2_3"
val_data_dir = "/content/FinalData/augmented_2_3"
num_train = 0
num_val = 0

for i in os.listdir(train_data_dir):
    if i.startswith("."):
        os.rmdir(os.path.join(train_data_dir, i))
    else:
        print(i,len(os.listdir(os.path.join(train_data_dir, i))))
        num_train += len(os.listdir(os.path.join(train_data_dir, i)))
        
for i in os.listdir(val_data_dir):
    if i.startswith("."):
        os.rmdir(os.path.join(val_data_dir, i))
    else:
        num_val += len(os.listdir(os.path.join(val_data_dir, i)))

img_width, img_height = 224, 224
if K.image_data_format() == 'channels_first':
    input_shape= (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [ ]:
from tensorflow.keras.applications.resnet50 import preprocess_input

from tensorflow.python.keras.applications.densenet import DenseNet121
from tensorflow.python.keras.applications.vgg16 import VGG16
from tensorflow.python.keras.applications.vgg19 import VGG19
from tensorflow.python.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications import ResNet101, ResNet50
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.applications.xception import Xception

base_model = ResNet50(input_shape=(224, 224, 3),
                                  weights='imagenet',
                                  include_top = False,
                                  pooling='avg')

for layer in base_model.layers:
    layer.trainable = False
x = base_model.output
x = Dense(1024)(x)
x = Activation('relu')(x)
x = Dense(512)(x)
x = Activation('relu')(x)
x = Dense(64)(x)
x = Activation('relu')(x)

predictions = Dense(5, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(loss = "categorical_crossentropy", optimizer = optimizers.Adam(lr=0.001), metrics = ['accuracy'])



In [ ]:
import fnmatch
from sklearn.utils import class_weight
y=[]
for i in range(5):
    y.extend([i for j in range(len(fnmatch.filter(os.listdir("/content/FinalData/final_training_data/"+str(i)),'*.jpg')))])
class_wgt = class_weight.compute_class_weight('balanced',np.unique(y),y)
class_dict = dict(zip(range(5),class_wgt))
print(class_dict)

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau
def get_callbacks():
    path_checkpoint ='checkpoint_keras'  
    log_dir='logs'   
    callback_checkpoint = ModelCheckpoint(filepath=path_checkpoint,
                                         monitor='accuracy',
                                         verbose=1,
                                         save_weights_only=False,
                                         save_best_only=True,
                                         mode='max',
                                         period=1)
    callback_early_stopping = EarlyStopping(monitor='accuracy',
                                           patience=5,
                                           verbose=1)
    callback_tensorboard = TensorBoard(log_dir=log_dir,
                                      histogram_freq=0,
                                      write_graph=False)
    callback_reduce_lr = ReduceLROnPlateau(monitor='accuracy',
                                          factor=0.1,
                                          min_lr=1e-4,
                                          patience=3,
                                          verbose=1)
    callbacks = [callback_checkpoint, callback_tensorboard, callback_reduce_lr]
    return callbacks


In [ ]:
batch_size = 128
epochs = 50                                                                
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   rotation_range=20,
                                  zoom_range=0.25,
                                  width_shift_range=0.3,
                                  height_shift_range=0.3,
                                  horizontal_flip=True,
                                  fill_mode="nearest")
train_generator = train_datagen.flow_from_directory(train_data_dir, 
                                                    target_size =(img_width, img_height),
                                                    batch_size = batch_size, class_mode = 'categorical',color_mode='rgb')
validation_generator = train_datagen.flow_from_directory( val_data_dir, 
                                                         target_size =(img_width, img_height),
                                                         batch_size = batch_size, class_mode ='categorical',color_mode='rgb') 
model.fit_generator(train_generator, 
                    steps_per_epoch = num_train // batch_size, 
                    epochs = epochs,
                    callbacks = get_callbacks(),class_weight= class_dict)


In [ ]:
history = model.history
import matplotlib.pyplot as plt
print(history.history)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.savefig("Accuracy_curves.png")
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.savefig("Loss_curves.png")

In [ ]:
import numpy
import sklearn.metrics as metrics
img_width, img_height = 224, 224
batch_size =1 
epochs = 100
test_set_dir = "/content/ValData"
val_set_dir = "/content/ValData"

num_test = len(os.listdir(test_set_dir))

print ("Number of images in test set: ", num_test)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = test_datagen.flow_from_directory(val_set_dir, 
                                                    target_size =(img_width, img_height), 
                                                    batch_size = batch_size, class_mode = 'categorical',color_mode='rgb',shuffle=False)
 
test_steps_per_epoch = numpy.math.ceil(test_generator.samples / test_generator.batch_size)

predictions = model.predict_generator(test_generator, steps = test_generator.samples)

predicted_classes = numpy.argmax(predictions, axis=1)

files = test_generator.filenames
#files=files.sort()

def validation():
    names = []
    for n in files:
        names.append(int(n.split("_")[1].split(".")[0]))
    acc = 0
    print(metrics.confusion_matrix(names,predicted_classes))
    print(metrics.accuracy_score(names,predicted_classes))
    print(metrics.classification_report(names,predicted_classes))

#validation()

output=[0,0,0,0,0]
print(np.bincount(predicted_classes))
import csv    
with open('predictions.csv', mode='w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    for x in predicted_classes:
        output2 = output[:]
        output2[int(x)] = 1
        csv_writer.writerow(output2)